In [1]:
using DataFrames, CSV

# I. Data Acquisition & Overview
https://www.kaggle.com/super13579/seir-model-pr-model-for-covid19-global-forecast

In [2]:
train_df = CSV.read("week2/train.csv")
first(train_df, 6)

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
,Int64,String⍰,String,Dates…,Float64,Float64
1,1,missing,Afghanistan,2020-01-22,0.0,0.0
2,2,missing,Afghanistan,2020-01-23,0.0,0.0
3,3,missing,Afghanistan,2020-01-24,0.0,0.0
4,4,missing,Afghanistan,2020-01-25,0.0,0.0
5,5,missing,Afghanistan,2020-01-26,0.0,0.0
6,6,missing,Afghanistan,2020-01-27,0.0,0.0


In [6]:
names(train_df)[1]

# info_df = pd.DataFrame(world_combined_df.dtypes)
# info_df['MissingVal'] = world_combined_df.isnull().sum()
# info_df['NUnique'] = world_combined_df.nunique()
# info_df['Count'] = world_combined_df.count()
# info_df['PercentMissingVal'] = info_df.apply(lambda info: round(info.MissingVal / (info.Count + info.MissingVal), 6), axis=1)
# round(info_df['MissingVal'] / info_df['Count'], 6)
# info_df = info_df.rename(columns = {0: 'DataType'})
# info_df

:Id

In [5]:
info_df = DataFrames(col = Strings.names(train_df))
info_df

MethodError: MethodError: objects of type Module are not callable